# Launch simulations on a slurm-based cluster


This script launches simulations on a SLURM cluster using submitit.

In [2]:
import submitit
import os
import numpy as np
import matplotlib.pyplot as plt

from run_RNN_pixel_simulation import run_rnn
from reduction import compute_timecourse_zpolicy, descriptive_zpol

## Local test runs

This script can submit these long-running jobs to the cluster. However, the next two cells show how to run locally.

In [ ]:
# Run a blob (real image) simulation localy
data_type = 'blob'
dt = .05 #.0005
lr = .1
gamma = .01
blob_scale = 50.

expt_folder = f"rnn_sim_res_{data_type}_lr_{lr}_bs_{blob_scale}_dt{dt}_gam{gamma}"
exptname = f"{expt_folder}/exp"

run_rnn(Ntrials=10000, data_type=data_type, blob_scale=blob_scale, lr=lr, dt=dt, gamma=gamma, exptname=exptname)

In [ ]:
# Run a Gaussian simulation locally
dt = .0005
lr=.1
gamma = .01
alpha=.05

run_rnn(data_type='gauss', exptname='test_gauss', lr=lr, dt=dt, gamma=gamma, alpha=alpha)

## Submit array jobs to cluster

In [44]:
# Submit array job to SLURM cluster for blob (real image) stimuli

executor = submitit.AutoExecutor(folder="log_exp")

executor.update_parameters(timeout_min=2*60*100, mem_gb=5, cpus_per_task=1, slurm_partition="cpu")

data_type = 'blob'
dt = .0005
lr = .1
gamma = .01
alpha = .05
blob_scale = 50.

expt_folder = f"rnn_sim_res2_{data_type}_lr_{lr}_bs_{blob_scale}_dt{dt}_gam{gamma}_al{alpha}"

if not os.path.exists(expt_folder):
    os.makedirs(expt_folder)

jobnames = []
with executor.batch():
    for rep in range(10):
        exptname = f"{expt_folder}/exp{rep}"
        jobnames.append(exptname)
        job = executor.submit(run_rnn, Ntrials=10000, data_type=data_type, blob_scale=blob_scale, lr=lr, dt=dt, gamma=gamma, alpha=alpha, exptname=exptname) 

In [ ]:
# Submit array job to SLURM cluster for Gaussian stimuli

executor = submitit.AutoExecutor(folder="log_exp")

executor.update_parameters(timeout_min=2*60, mem_gb=5, cpus_per_task=1, slurm_partition="cpu")

data_type = 'gauss'
dt = .0005
lr = .1
gamma = .01
alpha = .05
blob_scale = 50.

lr=.1
gamma = .01
alpha=.05
z0 = 30.
w0 = .0001

# Dataset parameters
co = 32
# For theory only
A = .82
ci = .01


expt_folder = f"rnn_sim_res2_{data_type}_lr_{lr}_dt{dt}_gam{gamma}_al{alpha}"

if not os.path.exists(expt_folder):
    os.makedirs(expt_folder)

jobnames = []
with executor.batch():
    for rep in range(50):
        exptname = f"{expt_folder}/exp{rep}"
        jobnames.append(exptname)
        job = executor.submit(run_rnn, Ntrials=10000, data_type=data_type, lr=lr, dt=dt, gamma=gamma, alpha=alpha, exptname=exptname) 

When these runs are complete, the notebook 'figure_RNN_reduction_comparison.ipynb' can read and plot the resulting comparisons